In [2]:
import pandas as pd
import math

In [3]:
RIDERS = pd.read_csv("../data/riders.csv")
RIDER_TEAMS = pd.read_csv("../data/rider_teams.csv")
MANAGERS = pd.read_csv("../data/managers.csv")
MANAGER_TEAMS = pd.read_csv("../data/manager_teams.csv")
POINTS_SYSTEM = pd.read_csv("../data/points_system.csv")
RACES = pd.read_csv("../data/races.csv")
RESULTS_2023 = pd.read_csv("../data/results/results_2023_full.csv")
RESULTS_2024 = pd.read_csv("../data/results/results_2024_full.csv")
RESULTS_2025 = pd.read_csv("../data/results/results_2025_full.csv")
CHEAPO_BANS = pd.read_csv("../data/cheapo_bans.csv")

In [4]:
# Riders owned by
RIDERS_OWNED_BY = MANAGER_TEAMS.groupby(["RiderName"])["ManagerName"].apply(", ".join).reset_index()

# Race categories
RACE_CATEGORIES = RACES[["RaceName_PCS", "RaceCategory"]]

# Relevant results
RESULTS_POINTS = RESULTS_2025[["RACE", "STAGE_ID", "STAGE_TYPE", "RANK", "RIDER"]]
RESULTS_POINTS = RESULTS_POINTS.rename(columns = {
    "RACE": "RaceName",
    "STAGE_ID": "StageID",
    "STAGE_TYPE": "StageType",
    "RANK": "Rank",
    "RIDER": "RiderName"
})
RESULTS_POINTS = RESULTS_POINTS.dropna(subset = ["StageID"])
RESULTS_POINTS = RESULTS_POINTS.loc[RESULTS_POINTS["StageType"] != "TTT"]

def convert_to_int(row):
    if row.isdigit():
        return int(row)
    else:
        return 0
    
def stage_or_gcresult(stage_id, race_category):
    if stage_id.startswith("stage"):
        return race_category.replace("Tour ", "Stage ")
    else:
        return race_category    

RESULTS_POINTS = pd.merge(RESULTS_POINTS, RACE_CATEGORIES, how = "inner", left_on = ["RaceName"], right_on = ["RaceName_PCS"])
RESULTS_POINTS["Rank"] = RESULTS_POINTS.apply(lambda row: convert_to_int(row["Rank"]), axis = 1)
RESULTS_POINTS["RaceCategory"] = RESULTS_POINTS.apply(lambda row: stage_or_gcresult(row["StageID"], row["RaceCategory"]), axis = 1)
RESULTS_POINTS = pd.merge(RESULTS_POINTS, POINTS_SYSTEM, how = "left", left_on = ["RaceCategory", "Rank"], right_on = ["RaceCategory", "RaceRank"])
RESULTS_POINTS = pd.merge(RESULTS_POINTS, RIDERS, how = "left", left_on = ["RiderName"], right_on = ["RiderName_PCS"])

# Rider points
RIDER_POINTS = RESULTS_POINTS.groupby(["RiderName_Zweeler"])["RacePoints"].sum().reset_index()

# Rider racedays
RIDER_RACEDAYS = RESULTS_POINTS.loc[RESULTS_POINTS["StageID"] != "gc"]
RIDER_RACEDAYS = RIDER_RACEDAYS.groupby(["RiderName_Zweeler"])["RiderName_Zweeler"].count().rename("Racedays").reset_index()

In [5]:
RIDER_OUTPUT = RIDERS

RIDER_OUTPUT = pd.merge(RIDER_OUTPUT, RIDER_RACEDAYS, how = "left", left_on = ["RiderName_Zweeler"], right_on = ["RiderName_Zweeler"])
RIDER_OUTPUT = pd.merge(RIDER_OUTPUT, RIDER_POINTS, how = "left", left_on = ["RiderName_Zweeler"], right_on = ["RiderName_Zweeler"])
RIDER_OUTPUT = pd.merge(RIDER_OUTPUT, RIDERS_OWNED_BY, how = "left", left_on = ["RiderName_Zweeler"], right_on = ["RiderName"])
RIDER_OUTPUT = pd.merge(RIDER_OUTPUT, CHEAPO_BANS, how = "left", left_on = ["RiderName_Zweeler"], right_on = ["RiderName"])

def ppm(points, millions):
    if math.isnan(points):
        return round(0, 2)
    else:
        return round(points / millions, 1)

def float_to_int(row):
    if math.isnan(row):
        return 0
    else:
        return int(row)

def cheapo_eligible(price, ban):
    if price <= 2.5 and ban == "":
        return "Ja"
    else:
        return ""

RIDER_OUTPUT["PPM"] = RIDER_OUTPUT.apply(lambda row: ppm(row["RacePoints"], row["RiderPrice"]), axis = 1)
RIDER_OUTPUT.RiderName_y = RIDER_OUTPUT.RiderName_y.fillna("")
RIDER_OUTPUT["Cheapo"] = RIDER_OUTPUT.apply(lambda row: cheapo_eligible(row["RiderPrice"], row["RiderName_y"]), axis = 1)
RIDER_OUTPUT["RacePoints"] = RIDER_OUTPUT.apply(lambda row: float_to_int(row["RacePoints"]), axis = 1)
RIDER_OUTPUT["Racedays"] = RIDER_OUTPUT.apply(lambda row: float_to_int(row["Racedays"]), axis = 1)
RIDER_OUTPUT.ManagerName = RIDER_OUTPUT.ManagerName.fillna("")

RIDER_OUTPUT = RIDER_OUTPUT[["RiderName_Zweeler", "RiderTeam", "RiderPrice", "Racedays", "RacePoints", "PPM", "Cheapo", "ManagerName"]]

RIDER_OUTPUT = RIDER_OUTPUT.rename(columns = {
    "RiderName_Zweeler": "Navn",
    "RiderTeam": "Hold",
    "RiderPrice": "Pris",
    "Racedays": "Løbsdage",
    "RacePoints": "Point",
    "PPM": "Profit",
    "Cheapo": "Cheapo",
    "ManagerName": "Managers"
})

# Still need to add:
# RACEDAYS TOTAL (Number from PCS)
# Points 2024 (calculated)
# Picked by number in game (need to scrape from Zweeler)

In [6]:
TEAMS_WITH_POINTS = pd.merge(MANAGER_TEAMS, RIDER_OUTPUT, how = "left", left_on = "RiderName", right_on = "Navn")
STANDINGS = TEAMS_WITH_POINTS.groupby(["ManagerName"])[["Pris", "Løbsdage", "Point"]].sum().reset_index()
STANDINGS["Point per løbsdag"] = round(STANDINGS["Point"] / STANDINGS["Løbsdage"], 2)

In [8]:
RIDER_OUTPUT.to_html("../outputtables/RytteroversigtTabel.html", table_id = "filterabletable", index = False)
